### Metadata

In [1]:
import random
import pandas as pd
import numpy as np
from sklearn import metrics

In [22]:
# Read Training data
metadata_df = pd.read_csv("data/Dev_Set/metadata.csv")
# Read Test data
metadata_test_df = pd.read_csv("data/Test_Set/metadata_test.csv")

# Convert the runtime into int
metadata_df["runtime"] = metadata_df["runtime"].apply(lambda x: x.split(" min")[0])
metadata_df["runtime"] = metadata_df["runtime"].astype(float)

# Features
metadata_x = metadata_df.drop("goodforairplane", axis=1)
# Target
metadata_Y = metadata_df["goodforairplane"]

# Same for test set
# Features
metadata_test_x = metadata_test_df.drop("goodforairplane", axis=1)
# Target
metadata_test_Y = metadata_test_df["goodforairplane"]

In [40]:
meta_x = metadata_x
meta_Y = metadata_Y
features_ind = np.array(list(range(0, len(meta_x.columns))))
num_features = len(features_ind)
num_selected_features = np.random.randint(1,num_features+1)
S1_ind = np.random.choice(features_ind, size=num_selected_features, replace = False)
np.sort(S1_ind)
#meta_x.iloc[:,[2,5,1]]
empty = np.array([])
empty

len(meta_x.columns)
len(metadata_test_x.columns)

100

In [41]:
np.random.seed(50)

def LVW(meta_x, meta_Y, meta_test_x, meta_test_Y, K, classifier):
    err = 0
    k = 0
    C = 100
    # Best features
    S = np.array([])
    # Create list of features
    while k < K:
        # Create a list of available features
        features_ind = np.array(list(range(0, len(meta_x.columns))))
        num_features = len(features_ind)
        # Generate how many features should be selected
        C1 = np.random.randint(1,num_features+1)
        # Randomly pick num_selected_features as subset
        S1_ind = np.random.choice(features_ind, size=C1, replace = False)
        # Sort it, because it looks nicer...
        S1_ind = np.sort(S1_ind)
        # Train the classifier
        classifier.fit(meta_x.iloc[:,S1_ind], meta_Y)
        # Predict the results
        pred = classifier.predict(meta_x.iloc[:,S1_ind])
        # Comptue f1 measure (since in the paper it is stated that they tune it based on the f1 score)
        f1 = metrics.f1_measure(meta_Y, pred)
        
        if (f1 < err or (f1 == err and C1 < C)):
            k = 0
            err = f1
            C = C1
            S = S1
        else:
            k = k+1
    # Test on testset
    classifier.fit(meta_x.iloc[:,S], meta_Y)
    #test_pred = classifier.predict()
    
    return S

,rated,g_Action,runtime
0,PG-13,1,102.0
1,R,0,105.0
2,R,0,141.0
3,PG-13,1,165.0
4,TV-MA,0,132.0
...,...,...,...
90,R,0,95.0
91,R,0,154.0
92,NaN,0,94.0
93,G,0,78.0
